In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI
import spacy
sp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

bitcoin = pd.read_csv("../data/raw/bitcoin_3_month.csv")
btc = pd.read_csv("../data/raw/btc_3_month.csv")
bitcoin.head()


/var/folders/09/339ff8256958pb0n3nhwztww0000gn/T/ipykernel_75516/4227697864.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bitcoin = pd.read_csv("../data/raw/bitcoin_3_month.csv")


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1454233293109534720,1454233293109534720,2021-10-29 19:46:50 EDT,2021-10-29,19:46:50,-500,963767159536209921,theblock__,The Block,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1454232278092107780,1454229634648793094,2021-10-29 19:42:48 EDT,2021-10-29,19:42:48,-500,823766058909761536,lynaldencontact,Lyn Alden,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Croesus_BTC', 'name': 'Croes...",NaN,NaN,NaN,NaN
2,1454231129804972042,1454231129804972042,2021-10-29 19:38:14 EDT,2021-10-29,19:38:14,-500,729548255470575616,eric_lombrozo,👑,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1454230807233581061,1454229634648793094,2021-10-29 19:36:57 EDT,2021-10-29,19:36:57,-500,823766058909761536,lynaldencontact,Lyn Alden,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'skwp', 'name': 'Yan Pritzker...",NaN,NaN,NaN,NaN
4,1454229932255719425,1454220157220364288,2021-10-29 19:33:28 EDT,2021-10-29,19:33:28,-500,14110443,muneeb,muneeb.btc,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'jack', 'name': 'jack⚡️', 'id...",NaN,NaN,NaN,NaN


In [2]:
# select some column
bitcoin = bitcoin[['id','date','time','tweet','language','replies_count','retweets_count', 'likes_count']]
btc = btc[['id','created_at','tweet','language','replies_count','retweets_count', 'likes_count']]
print(bitcoin.shape)
print(btc.shape)

# # drop ADS???
bitcoin_drop = bitcoin.drop_duplicates(keep=False)
btc_drop = btc.drop_duplicates(keep=False)
print(bitcoin_drop.shape)
print(btc_drop.shape)

# bitcoin_btc = bitcoin_drop.merge(btc_drop, on='id', how='outer', indicator='btc_status')
bitcoin_btc  = pd.concat([bitcoin_drop, btc_drop],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=False,
)
bitcoin_btc = bitcoin_btc.drop_duplicates(subset='id')
print(bitcoin_btc.shape)
bitcoin_btc

(46566, 8)
(45581, 7)
(41328, 8)
(596, 7)
(41820, 9)


,id,date,time,tweet,language,replies_count,retweets_count,likes_count,created_at
172,1454132785908797441,2021-10-29,13:07:27,7/ Snow Crash (1992) predicted digital currenc...,en,2,8,55,NaN
173,1454132298983649295,2021-10-29,13:05:31,"For one day only, I am accepting bitcoin from ...",en,0,0,0,NaN
174,1454132043739185162,2021-10-29,13:04:30,@gator_bitcoin Do you own floki ?,en,0,0,1,NaN
175,1454131968132648971,2021-10-29,13:04:12,@PizzaAtlanta @JoePompliano @BitcoinPizza_io @...,en,0,0,4,NaN
176,1454131699290357764,2021-10-29,13:03:08,@jacque_davis I would be blind with panic if I...,en,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...
45238,1486874424364183556,NaN,NaN,"@WellbourneM Once the $btc price stabilizes, A...",en,2,0,5,2022-01-27 20:31:02 EST
45239,1486871174164324352,NaN,NaN,@JuanSGuarnizo TO THE MOON $BTC $ETH $USDT $EL...,en,3,3,395,2022-01-27 20:18:07 EST
45240,1486868220413984771,NaN,NaN,@BosRec I fought this kind of FUD from BTC max...,en,0,0,21,2022-01-27 20:06:23 EST
45241,1486865585774678016,NaN,NaN,BREAKING: Candidate for Governor of #Texas lay...,en,8,0,9,2022-01-27 19:55:54 EST


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chayisara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
# language EN only
df = bitcoin_btc[bitcoin_btc['language'] == 'en'].reset_index()
print(df["tweet"].shape)
df['tweet'] = df['tweet'].astype(str)
tweet = df['tweet']

(41773,)


In [25]:
# apply all lower case
df_lower = pd.DataFrame(df["tweet"].str.lower())
# remove URL "https://t.co/"
new_url = [re.sub(r"https://t.co/[A-Za-z0-9]+", " ", tweet) for tweet in df_lower.tweet]
# remove mention
new_mention = [re.sub(r"@[A-Za-z0-9!#%&*;_\$\.]+", " ", tweet) for tweet in new_url]

In [26]:
for i in new_mention:
    spacy_tweet = nlp(i)
#     print(type(spacy_tweet))
    for one_tok in spacy_tweet.ents:
        print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: 7/; NER tag: DATE
Entity: 1992; NER tag: DATE
Entity: one day; NER tag: DATE
Entity: $200 million; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: first; NER tag: ORDINAL
Entity: #; NER tag: CARDINAL
Entity: 20,000; NER tag: MONEY
Entity: today; NER tag: DATE
Entity: 100; NER tag: MONEY
Entity: 250; NER tag: MONEY
Entity: 500; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: 10/29; NER tag: CARDINAL
Entity: halloween; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: this year; NER tag: DATE
Entity: benchmark; NER tag: PERSON
Entity: 0.25%; NER tag: PERCENT
Entity: 2022; NER tag: DATE
Entity: 21; NER tag: CARDINAL
Entity: 3rd; NER tag: ORDINAL
Entity: #; NER tag: CARDINAL
Entity: 2018; NER tag: DATE
Entity: 10th; NER tag: ORDINAL
Entity: halloween; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: close to $63k; NER tag: MONEY
Entity: #; NER tag: ORG
Entity: nearly 6%; NER tag: PERCENT
Entity: 4400; NER tag: MONEY
Entity: #; NER tag: CARDIN

Entity: $1 trillion; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: 100,000; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: daily; NER tag: DATE
Entity: 29; NER tag: CARDINAL
Entity: #kitconews #; NER tag: MONEY
Entity: #finance #; NER tag: MONEY
Entity: #; NER tag: ORG
Entity: milegi dobara; NER tag: PERSON
Entity: another 6,500; NER tag: CARDINAL
Entity: chinese; NER tag: NORP
Entity: august; NER tag: DATE
Entity: january; NER tag: DATE
Entity: 2014; NER tag: DATE
Entity: tomorrow; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: october; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: 500%; NER tag: PERCENT
Entity: this year; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: the past 6 months; NER tag: DATE
Entity: 3-year; NER tag: DATE
Entity: one; NER tag: CARDINAL
Entity: #; NER tag: DATE
Entity: #; NER tag: CARDINAL
Entity: 🤩; NER tag: ORG
Entity: #; NER tag: ORG
Entity: #; NER

Entity: congress; NER tag: ORG
Entity: pawan khera; NER tag: PERSON
Entity: 11am; NER tag: TIME
Entity: today; NER tag: DATE
Entity: delhi; NER tag: GPE
Entity: #bengaluru     ; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: congress; NER tag: MONEY
Entity: 4%; NER tag: PERCENT
Entity: the day; NER tag: DATE
Entity: 58,850; NER tag: MONEY
Entity: 4%; NER tag: PERCENT
Entity: today; NER tag: DATE
Entity: daily; NER tag: DATE
Entity: 25-year-old; NER tag: DATE
Entity: cordacon2021; NER tag: MONEY
Entity: square inc.'s; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: us; NER tag: GPE
Entity: 600; NER tag: CARDINAL
Entity: tampa; NER tag: GPE
Entity: buccaneers; NER tag: NORP
Entity: tom brady; NER tag: PERSON
Entity: kennedy; NER tag: PERSON
Entity: 600th; NER tag: ORDINAL
Entity: us; NER tag: GPE
Entity: #globalmarkets |; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: asian; NER tag: NORP
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: ORG
Entity

Entity: #; NER tag: CARDINAL
Entity: 61702.40; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: 4317.92; NER tag: CARDINAL
Entity: -2.03%; NER tag: PERCENT
Entity: #; NER tag: ORG
Entity: 1.9814 -1.34%; NER tag: PERCENT
Entity: #; NER tag: CARDINAL
Entity: 4.4461 -1.61%; NER tag: PERCENT
Entity: #; NER tag: CARDINAL
Entity: 192.38 -1.17%; NER tag: PERCENT
Entity: 0.3587; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: 10; NER tag: CARDINAL
Entity: 23786; NER tag: CARDINAL
Entity: -1.93%; NER tag: PERCENT
Entity: #; NER tag: ORG
Entity: $eth #; NER tag: MONEY
Entity: #xlm #; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: etfs; NER tag: ORG
Entity: china; NER tag: GPE
Entity: el salvador's; NER tag: GPE
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: third; NER tag: ORDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: mike mcglone; NER tag: PERSON
Entity: #; NER tag: CARDINAL
Entity: 100k; NER tag: MONEY
Entity: fgu; NER tag: GPE
E

Entity: #; NER tag: CARDINAL
Entity: 🚀; NER tag: ORG
Entity: 🚀; NER tag: CARDINAL
Entity: jack dorsey; NER tag: PERSON
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: ORG
Entity: daily; NER tag: DATE
Entity: today; NER tag: DATE
Entity: 200; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: btc daily; NER tag: PERSON
Entity: 63,000; NER tag: MONEY
Entity: 64,500; NER tag: MONEY
Entity: intraday; NER tag: TIME
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: ORG
Entity: australian; NER tag: NORP
Entity: #; NER tag: CARDINAL
Entity: today; NER tag: DATE
Entity: #; NER tag: ORG
Entity: ’s 13th; NER tag: DATE
Entity: a single $; NER tag: MONEY
Entity: 0.0008; NER tag: MONEY
Entity: a single #; NER tag: MONEY
Entity: one day; NER tag: DATE
Entity: 🚀; NER tag: ORG
Entity: 🚀; NER tag: PRODUCT
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: 3rd; NER tag: ORDINAL
Entity: #; NER tag: CARDI

...

Entity: #; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: k'taka; NER tag: GPE
Entity: k'taka; NER tag: GPE
Entity: two; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: jpmorgan; NER tag: PERSON
Entity: 4; NER tag: CARDINAL
Entity: 60/40; NER tag: CARDINAL
Entity: today; NER tag: DATE
Entity: #trading #; NER tag: MONEY
Entity: s2f; NER tag: ORG
Entity: just a few days ago; NER tag: DATE
Entity: 5%; NER tag: PERCENT
Entity: sec; NER tag: ORG
Entity: 😎 smart; NER tag: PERSON
Entity: 100,000; NER tag: MONEY
Entity: 5,000; NER tag: MONEY
Entity: 100,000; NER tag: MONEY
Entity: 5,000; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: 90k; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: sec; NER tag: ORG
Entity: u.s.; NER tag: ORG
Entity: sec; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: adam aron; NER tag: PERSON
Entity: 98k; NER tag: DATE
Entity: december 2; NER tag: DATE
Entity: 59,

Entity: #; NER tag: CARDINAL
Entity: 2.0; NER tag: CARDINAL
Entity: el salvador's; NER tag: GPE
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: murakami &amp; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: sec; NER tag: ORG
Entity: now; NER tag: DATE
Entity: more than 2 hours; NER tag: TIME
Entity: the past five years; NER tag: DATE
Entity: el salvador; NER tag: GPE
Entity: dubnikov; NER tag: PERSON
Entity: more than $400,000; NER tag: MONEY
Entity: millions; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: dutch; NER tag: NORP
Entity: el salvador; NER tag: GPE
Entity: 🇸🇻; NER tag: PERSON
Entity: #; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: today; NER tag: DATE
Entity: congress; NER tag: ORG
Entity: sec; NER tag: ORG
Entity: #; NER tag: CARDINAL
Entity: relai nice weekend; NER tag: DATE
Entity: etfs; NER tag: ORG
Entity: sec; NER tag: ORG
Entity: 5; NER tag: CARDINAL
Entity: 191,284; NER tag: CARDINAL
Entity: #; NER tag: C

Entity: #; NER tag: CARDINAL
Entity: first; NER tag: ORDINAL
Entity: 🌎; NER tag: WORK_OF_ART
Entity: #; NER tag: CARDINAL
Entity: abuelas; NER tag: PERSON
Entity: 🤯; NER tag: ORG
Entity: 🤯; NER tag: WORK_OF_ART
Entity: #; NER tag: CARDINAL
Entity: 24,000; NER tag: MONEY
Entity: 200; NER tag: CARDINAL
Entity: 160k; NER tag: MONEY
Entity: the last decade; NER tag: DATE
Entity: roughly 28%; NER tag: PERCENT
Entity: 99.996%; NER tag: PERCENT
Entity: crypto &amp; NER tag: ORG
Entity: #; NER tag: MONEY
Entity: #eth #; NER tag: MONEY
Entity: the week   ; NER tag: DATE
Entity: weekly; NER tag: DATE
Entity: 60k; NER tag: CARDINAL
Entity: 60k; NER tag: CARDINAL
Entity: 53k; NER tag: DATE
Entity: europe; NER tag: LOC
Entity: intraday; NER tag: TIME
Entity: 65080; NER tag: CARDINAL
Entity: 65,000; NER tag: CARDINAL
Entity: at least 250,000; NER tag: CARDINAL
Entity: #; NER tag: CARDINAL
Entity: $xlm - $; NER tag: MONEY
Entity: 46; NER tag: MONEY
Entity: #; NER tag: CARDINAL
Entity: #; NER tag: CAR

Entity: #; NER tag: CARDINAL


KeyboardInterrupt: 

In [29]:
# remove # ,turning hashtags to the typical words.
new_symbol = [re.sub(r"\W+", " ", tweet) for tweet in new_mention]
# remove 2-character words
new_twochar = [re.sub(r"\b[a-z]{1,2}\b", " ", tweet) for tweet in new_symbol]

In [30]:
# remove stopword
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

new_stopwords = remove_stop_words(new_twochar)

In [31]:
# reduce character sequences >3 to 3
sequence_pattern = re.compile(r"(\w)\1+")
def repl(matchObj):
   char = matchObj.group(1)
   return "%s%s" % (char, char)

new_sequence = [re.sub(sequence_pattern, repl, tweet) for tweet in new_stopwords]

In [32]:
# replace 2 or more spaces with a single space.
new_space = [re.sub(r"\s+", " ", tweet) for tweet in new_sequence]
# remove spaces front and back
new_space_end = [re.sub(r"^\s+|\s$", "", tweet) for tweet in new_space]
new_space_end[4]

'would blind panic realized worth stories people lost password need unlock like 200 million bitcoin god run massive therapy bill'

In [33]:
sent_obj = SentimentIntensityAnalyzer()
print(type(sent_obj))

sent_data = new_space_end
sentiment_example = []
for i in sent_data:
    sentiment_example.append(sent_obj.polarity_scores(i))

<class 'vaderSentiment.vaderSentiment.SentimentIntensityAnalyzer'>


In [34]:
# sent_obj.lexicon
neg = [sent_obj.lexicon[k] for k in sent_obj.lexicon if sent_obj.lexicon[k] < 0]
np.mean(neg)
pos = [sent_obj.lexicon[k] for k in sent_obj.lexicon if sent_obj.lexicon[k] > 0]
np.mean(pos)

# sent_obj.lexicon['']

-1.552026864955625

1.5321845969433623

In [35]:
pest_words = {
    'up': 2.0,
    'down': -2.0,
    'green' : 2.0,
    'red' : -2.0,
    'bull' : 2.0,
    'bear' : -2.0,
    'buy' : 2.0,
    'bought' : 2.0,
    'sell' : -2.0,
    'sold' : -2.0,
    'moon': 2
}

new_si = SentimentIntensityAnalyzer()
new_si.lexicon.update(pest_words)

sentiment_update = []
for i in sent_data:
    sentiment_update.append(sent_obj.polarity_scores(i))
sentiment_update

[{'neg': 0.251, 'neu': 0.512, 'pos': 0.237, 'compound': -0.0772},
 {'neg': 0.123, 'neu': 0.71, 'pos': 0.168, 'compound': 0.1779},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6369},
 {'neg': 0.288, 'neu': 0.486, 'pos': 0.226, 'compound': -0.4215},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.795, 'pos': 0.205, 'compound': 0.4767},
 {'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'compound': -0.1695},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.8555},
 {'neg': 0.037, 'neu': 0.488, 'pos': 0.476, 'compound': 0.9371},
 {'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.4019},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.237, 'neu': 0.536, 'pos': 0.228, 'compound': -0.0516},
 {'neg': 0.0, 'neu': 0.812, 'pos': 0.188, 'compound': 0.5267},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu

In [36]:
all_compound = [one_tok['compound'] for one_tok in sentiment_update]
temp = pd.DataFrame(all_compound)
df['compound'] = temp
df[['tweet', 'compound']]

,tweet,compound
0,7/ Snow Crash (1992) predicted digital currenc...,-0.0772
1,"For one day only, I am accepting bitcoin from ...",0.1779
2,@gator_bitcoin Do you own floki ?,0.0000
3,@PizzaAtlanta @JoePompliano @BitcoinPizza_io @...,0.6369
4,@jacque_davis I would be blind with panic if I...,-0.4215
...,...,...
41768,"@WellbourneM Once the $btc price stabilizes, A...",0.0000
41769,@JuanSGuarnizo TO THE MOON $BTC $ETH $USDT $EL...,0.0000
41770,@BosRec I fought this kind of FUD from BTC max...,-0.0516
41771,BREAKING: Candidate for Governor of #Texas lay...,0.1280


In [37]:
# weight retweet
df['compound'] = df['compound'].astype(float)
compound_weight = []
def get_weight_score(row):
    famous_score = row['replies_count']*1/3 + row['retweets_count']*1/3 + row['likes_count']*1/3
    if (famous_score < 50):
        compound_weight.append(row['compound'])
    elif (famous_score > 50) and (famous_score < 200):
        compound_weight.append(row['compound']*2)
    elif (famous_score > 200) and (famous_score < 500):
        compound_weight.append(row['compound']*4)
    elif (famous_score > 500) and (famous_score < 999):
        compound_weight.append(row['compound']*8)
    elif (famous_score > 1000) and (famous_score < 3000):
        compound_weight.append(row['compound']*16)
    elif (famous_score > 3000) and (famous_score < 9999):
        compound_weight.append(row['compound']*32)
    else:
        compound_weight.append(row['compound']*64)

df.apply(get_weight_score, axis=1)
df['compound_weight'] = compound_weight
df['compound_weight'] 



0        None
1        None
2        None
3        None
4        None
         ... 
41768    None
41769    None
41770    None
41771    None
41772    None
Length: 41773, dtype: object

0       -0.0772
1        0.1779
2        0.0000
3        0.6369
4       -0.4215
          ...  
41768    0.0000
41769    0.0000
41770   -0.0516
41771    0.1280
41772    0.0000
Name: compound_weight, Length: 41773, dtype: float64

In [38]:
# # create date-time column
df_bydate = df.groupby(['date']).agg({"compound_weight" : "mean"}).reset_index()
df_bydate[:20]

,date,compound_weight
0,2021-10-28,0.937869
1,2021-10-29,0.672121
2,2021-10-30,0.712180
3,2021-10-31,0.506298
4,2021-11-01,0.410790
5,2021-11-02,0.493032
6,2021-11-03,0.343299
7,2021-11-04,0.378746
8,2021-11-05,0.321440
9,2021-11-06,0.672677


In [39]:
# Stremming ex hom/home/homey
# group compound words
# word mattrix??
# add stopwords

In [49]:
all_money
def get_money(one_string):
    
    tagged_str = nlp(one_string) 
    all_money = [one_tok.text for one_tok in tagged_str.ents 
                  if one_tok.label_ == "MONEY"]
    return(all_money)

all_money = [get_money(one_string) for one_string in new_mention]

[[],
 [],
 [],
 [],
 ['$200 million'],
 [],
 [],
 [],
 [],
 ['20,000', '100', '250', '500'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['close to $63k', '4400'],
 [],
 [],
 ['$fb $'],
 [],
 ['600'],
 [],
 [],
 ['about $25 million'],
 [],
 ['june - $aapl'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['43'],
 [],
 ['$500 million'],
 [],
 [],
 [],
 [],
 ['100k'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#moneyheist #'],
 [],
 [],
 [],
 ['3000'],
 [],
 ['more than $1.3 billion', '$65 million'],
 [],
 [],
 ['62349.20 +1.55%', '#btc $eth #bch', '#xlm #'],
 ['58k', '63k'],
 [],
 [],
 [],
 ['$300 million'],
 ['$300 million'],
 [],
 [],
 [],
 ['500'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['#eth #'],
 [],
 [],
 [],
 [],
 [],
 ['100'],
 [],
 ['$shb #'],
 [],
 [],
 [],
 [],
 [],
 ['#'],
 [],
 ['$22 million', '$3 billion', '$1.16 trillion', '$1.09 trillion'],
 [],
 [],
 [],
 [],
 [],
 ['millions of dollars'],
 [],
 [],
 ['20,000'],
 ['phun'],
 [],
 [],
 [],
 [],
 ['#goodovere

In [54]:
for i in all_money:
    for j in i: 
        if (j > '30000') or (j < '100000'):
            print()
        
all_money

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['500'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['$1.09 trillion'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],